In [1]:
import requests
import pandas as pd
from datetime import datetime

# Your team code mapping based on the image
TEAM_CODE_MAPPING = {
    'Arsenal': 3,
    'Aston Villa': 7,
    'Bournemouth': 91,
    'Brentford': 94,
    'Brighton': 36,
    'Burnley': 90,
    'Chelsea': 8,
    'Crystal Palace': 31,
    'Everton': 11,
    'Fulham': 54,
    'Liverpool': 14,
    'Man City': 43,
    'Man Utd': 1,
    'Newcastle': 4,
    'Nott\'m Forest': 17,
    'Spurs': 6,
    'West Ham': 21,
    'Wolves': 39,
    'Leeds': 2,
    'Ipswich': 40,
    'Leicester': 13,
    'Southampton': 20,
    'Luton': 102,
    'Sheffield Utd': 49,
    'Sunderland': 56
}

def get_fpl_fixtures():
    """Fetch fixtures from FPL API"""
    print("Fetching fixtures from FPL API...")
    
    # Get fixtures
    fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
    fixtures_response = requests.get(fixtures_url)
    fixtures_data = fixtures_response.json()
    
    # Get teams to map IDs to names
    bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    bootstrap_response = requests.get(bootstrap_url)
    bootstrap_data = bootstrap_response.json()
    
    # Create team ID to name mapping from FPL
    fpl_teams = {team['id']: team['name'] for team in bootstrap_data['teams']}
    
    print(f"Found {len(fixtures_data)} fixtures")
    return fixtures_data, fpl_teams

def create_fixtures_csv(fixtures_data, fpl_teams):
    """Convert fixtures to CSV format with your team codes"""
    
    fixtures_list = []
    
    for fixture in fixtures_data:
        # Only include fixtures that haven't been played yet
        if not fixture['finished']:
            home_team_fpl_name = fpl_teams[fixture['team_h']]
            away_team_fpl_name = fpl_teams[fixture['team_a']]
            
            # Try to match FPL names to your codes
            # Handle name variations
            home_team_name = map_team_name(home_team_fpl_name)
            away_team_name = map_team_name(away_team_fpl_name)
            
            # Get your custom codes
            home_code = TEAM_CODE_MAPPING.get(home_team_name, None)
            away_code = TEAM_CODE_MAPPING.get(away_team_name, None)
            
            if home_code and away_code:  # Only include if we have codes
                fixture_dict = {
                    'gameweek': fixture['event'],
                    'home_team': home_team_name,
                    'away_team': away_team_name,
                    'homeid': home_code,
                    'awayid': away_code,
                    'date': fixture['kickoff_time'],
                    'home_strength': 3,  # Default, adjust based on your data
                    'away_strength': 3   # Default, adjust based on your data
                }
                fixtures_list.append(fixture_dict)
    
    # Create DataFrame
    df = pd.DataFrame(fixtures_list)
    
    # Sort by date
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    
    # Format date
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    return df

def map_team_name(fpl_name):
    """Map FPL team names to your naming convention"""
    name_mapping = {
        'Arsenal': 'Arsenal',
        'Aston Villa': 'Aston Villa',
        'Bournemouth': 'Bournemouth',
        'Brentford': 'Brentford',
        'Brighton and Hove Albion': 'Brighton',
        'Brighton': 'Brighton',
        'Burnley': 'Burnley',
        'Chelsea': 'Chelsea',
        'Crystal Palace': 'Crystal Palace',
        'Everton': 'Everton',
        'Fulham': 'Fulham',
        'Liverpool': 'Liverpool',
        'Manchester City': 'Man City',
        'Man City': 'Man City',
        'Manchester United': 'Man Utd',
        'Man Utd': 'Man Utd',
        'Newcastle United': 'Newcastle',
        'Newcastle': 'Newcastle',
        'Nottingham Forest': 'Nott\'m Forest',
        'Nott\'m Forest': 'Nott\'m Forest',
        'Tottenham Hotspur': 'Spurs',
        'Tottenham': 'Spurs',
        'Spurs': 'Spurs',
        'West Ham United': 'West Ham',
        'West Ham': 'West Ham',
        'Wolverhampton Wanderers': 'Wolves',
        'Wolves': 'Wolves',
        'Leeds United': 'Leeds',
        'Leeds': 'Leeds',
        'Ipswich Town': 'Ipswich',
        'Ipswich': 'Ipswich',
        'Leicester City': 'Leicester',
        'Leicester': 'Leicester',
        'Southampton': 'Southampton',
        'Luton Town': 'Luton',
        'Luton': 'Luton',
        'Sheffield United': 'Sheffield Utd',
        'Sheffield Utd': 'Sheffield Utd',
        'Sunderland': 'Sunderland'
    }
    
    return name_mapping.get(fpl_name, fpl_name)

def main():
    try:
        # Fetch data
        fixtures_data, fpl_teams = get_fpl_fixtures()
        
        # Create DataFrame
        df = create_fixtures_csv(fixtures_data, fpl_teams)
        
        # Save to CSV
        output_file = 'premier_league_fixtures.csv'
        df.to_csv(output_file, index=False)
        
        print(f"\n✅ Success! Created {output_file}")
        print(f"📊 Total fixtures: {len(df)}")
        print(f"🎯 Gameweeks: {df['gameweek'].min()} to {df['gameweek'].max()}")
        print(f"\nFirst few rows:")
        print(df.head(10).to_string())
        
        # Show any teams that might be missing
        print("\n⚠️  Teams in your code mapping:")
        for team in TEAM_CODE_MAPPING.keys():
            print(f"  - {team}")
            
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Fetching fixtures from FPL API...
Found 380 fixtures

✅ Success! Created premier_league_fixtures.csv
📊 Total fixtures: 310
🎯 Gameweeks: 8 to 38

First few rows:
   gameweek       home_team    away_team  homeid  awayid                 date  home_strength  away_strength
0         8   Nott'm Forest      Chelsea      17       8  2025-10-18 11:30:00              3              3
1         8        Brighton    Newcastle      36       4  2025-10-18 14:00:00              3              3
2         8         Burnley        Leeds      90       2  2025-10-18 14:00:00              3              3
3         8  Crystal Palace  Bournemouth      31      91  2025-10-18 14:00:00              3              3
4         8        Man City      Everton      43      11  2025-10-18 14:00:00              3              3
5         8      Sunderland       Wolves      56      39  2025-10-18 14:00:00              3              3
6         8          Fulham      Arsenal      54       3  2025-10-18 16:30:00      